In [1]:
#@title set up virtual display

from pyvirtualdisplay import Display

display = Display(visible=0, size=(1400, 900))
display.start()


# LunarLander with DQN 실습



## TODO

 이 파일은 학습을 실행하기 위한 코드입니다. 
 코드를 실행하기 위해서는 아래의 파일에서 다음 함수 #TODO 부분을 수정하셔야 합니다.
 
 - dqn_agent.py
     step_env()
     train()
 - dqn.py
     update()
 - core/dqn_utils.py 
     ReplayBuffer.write() : multi-step
 

## Run DQN

In [1]:
#imports library

import os
import time

from core.dqn_utils import get_env_kwargs

from rl_trainer import RL_Trainer
from dqn_agent import DQNAgent

%load_ext autoreload
%autoreload 2

In [2]:
#arguments (hyperparameter)

class Args:

  def __getitem__(self, key):
    return getattr(self, key)

  def __setitem__(self, key, val):
    setattr(self, key, val)

  def __contains__(self, key):
    return hasattr(self, key)

  env_name = 'LunarLander-v3' 
  exp_name = '_dqn' 

  ep_len = 200 
  num_timesteps = 150000 # 학습할 총 step 수

  #batches and steps
  batch_size = 128 # 학습에 사용할 batch_size

  num_agent_train_steps_per_iter = 1 # step 마다 network를 update하는 횟수

  # Advanced DQN
  double_q = False 
  n_step = 1

  save_params = False 
  no_gpu = False 
  which_gpu = 0 # 사용할 GPU number
  seed = 1 # 환경에 적용되는 seed

  video_log_freq =  -1 
  scalar_log_freq =  20000


args = Args()
args['train_batch_size'] = args['batch_size']

In [3]:
# 실험 결과 logging 설정
data_path = './data'

if not (os.path.exists(data_path)):
    os.makedirs(data_path)

logdir = 'day2_' + args.exp_name + '_LunarLander_seed'+ str(args.seed) + '_step' + str(args.n_step)
logdir = os.path.join(data_path, logdir)
args['logdir'] = logdir
if not(os.path.exists(logdir)):
    os.makedirs(logdir)

print("LOGGING TO: ", logdir)


LOGGING TO:  ./data/day2__dqn_LunarLander_seed1_step1


In [4]:
#@title Define Q-function trainer

class Q_Trainer(object):
    def __init__(self, params):
        self.params = params

        train_args = {
            'num_agent_train_steps_per_iter': params['num_agent_train_steps_per_iter'],
            'train_batch_size': params['batch_size'],
            'double_q': params['double_q'],
        }

        env_args = get_env_kwargs(params['env_name'])
        env_args['num_timesteps'] = params['num_timesteps']

        for k, v in env_args.items():
          params[k] = v

        self.params['agent_class'] = DQNAgent
        self.params['agent_params'] = params
        self.params['train_batch_size'] = params['batch_size']
        self.params['env_wrappers'] = env_args['env_wrappers']

        self.rl_trainer = RL_Trainer(self.params)

    def run_training_loop(self):
        self.rl_trainer.run_training_loop(
            self.params['num_timesteps'],
            collect_policy = self.rl_trainer.agent.dqn,
            eval_policy = self.rl_trainer.agent.dqn,
            )

In [ ]:
#run training

trainer = Q_Trainer(args)
trainer.run_training_loop()

########################
logging outputs to  ./data/day2__dqn_LunarLander_seed1_step1
########################
GPU not detected. Defaulting to CPU.
if 'env_wrappers' in self.params:
ob_dim = Box(-inf, inf, (9,), float32), ac_dim = 6


/Users/air/anaconda3/envs/sds/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
/Users/air/anaconda3/envs/sds/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Environment '<class 'envs.lunar_lander.LunarLander'>' has deprecated methods '_step' and '_reset' rather than 'step' and 'reset'. Compatibility code invoked. Set _gym_disable_underscore_compat = True to disable this behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


[ cs285/envs/box2d/lunar_lander ] No display found; rendering is disabled


/Users/air/anaconda3/envs/sds/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Tried to pass invalid video frame, marking as broken: Your frame has data type uint64, but we require uint8 (i.e. RGB values from 0-255).
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
  0%|          | 0/150000 [00:00<?, ?it/s]


********** Iteration 0 ************

Training agent...

Beginning logging procedure...
Timestep 1
Num Episodes 0
mean reward (100 episodes) nan
best mean reward -inf
running time 0.213690


...

/Users/air/anaconda3/envs/sds/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Tried to pass invalid video frame, marking as broken: Your frame has data type uint64, but we require uint8 (i.e. RGB values from 0-255).
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
 40%|███▉      | 59987/150000 [04:56<11:46, 127.45it/s]


********** Iteration 60000 ************

Training agent...

Beginning logging procedure...
Timestep 60001
Num Episodes 104
Grounded rate(%) = 20.19
Success rate(%) = 9.62
mean reward (100 episodes) -169.460662
best mean reward -169.460662
running time 296.981511


 40%|████      | 60018/150000 [04:56<10:55, 137.23it/s]

	Train_EnvstepsSoFar : 60001
	Train_EpisodeSoFar : 104
	Num_Episode_Grounded : 21
	Num_Episode_Grounded_at_site : 10
	Train_AverageReturn : -169.4606618056566
	Train_BestReturn : -169.4606618056566
	TimeSinceStart : 296.98151111602783
	Training Loss : 0.3812907040119171
Done logging...




 49%|████▉     | 73607/150000 [06:10<05:24, 235.10it/s]

In [8]:
from gym.wrappers import Monitor
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay

## modified from https://colab.research.google.com/drive/1flu31ulJlgiRL1dnN2ir8wGh9p7Zij2t#scrollTo=TCelFzWY9MBI

def show_video():
  mp4list = glob.glob(logdir + '/gym/*.mp4')
  print(logdir + '/gym/*.mp4')
  print(mp4list)
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")

show_video()

./data/hw3_q3_dqn_LunarLander-v3/gym/*.mp4
['./data/hw3_q3_dqn_LunarLander-v3/gym/openaigym.video.0.690.video000000.mp4']


In [7]:
#@markdown You can visualize your runs with tensorboard from within the notebook

## requires tensorflow==2.3.0
%load_ext tensorboard
%tensorboard --logdir .data/hw3_q3_dqn_LunarLander-v3/

In [3]:
#@title apt install requirements

#@markdown Run each section with Shift+Enter

#@markdown Double-click on section headers to show code.

!apt update 
!apt install -y --no-install-recommends \
        build-essential \
        curl \
        git \
        gnupg2 \
        make \
        cmake \
        ffmpeg \
        swig \
        libz-dev \
        unzip \
        zlib1g-dev \
        libglfw3 \
        libglfw3-dev \
        libxrandr2 \
        libxinerama-dev \
        libxi6 \
        libxcursor-dev \
        libgl1-mesa-dev \
        libgl1-mesa-glx \
        libglew-dev \
        libosmesa6-dev \
        lsb-release \
        ack-grep \
        patchelf \
        wget \
        xpra \
        xserver-xorg-dev \
        xvfb \
        python-opengl \
        ffmpeg > /dev/null 2>&1

#!pip install opencv-python==3.4.0.12

The operation couldn’t be completed. Unable to locate a Java Runtime that supports apt.
Please visit http://www.java.com for information on installing Java.

